In [17]:
import base64
import copy 
import json
import os 
import tempfile
import time
import fitz  
from fpdf import FPDF
from IPython.display import Markdown, display
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_core.messages import HumanMessage
from langchain_experimental.text_splitter import SemanticChunker
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from langchain_text_splitters import RecursiveCharacterTextSplitter
from mistralai import Mistral, DocumentURLChunk, ImageURLChunk, TextChunk
from mistralai.models import OCRResponse
from pathlib import Path
from pydantic import BaseModel
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, SparseVectorParams, VectorParams
from semantic_router.encoders import OpenAIEncoder
import concurrent.futures

In [19]:
file_path = "/home/hashim/multimodal-agentic-rag/data/1. Annual Report 2023-24.pdf"
loader = UnstructuredPDFLoader(file_path)
docs = loader.load()

Cannot set gray non-stroke color because /'P236' is an invalid float value
Cannot set gray non-stroke color because /'P242' is an invalid float value
Cannot set gray non-stroke color because /'P248' is an invalid float value
Cannot set gray non-stroke color because /'P254' is an invalid float value
Cannot set gray non-stroke color because /'P260' is an invalid float value
Cannot set gray non-stroke color because /'P268' is an invalid float value
Cannot set gray non-stroke color because /'P274' is an invalid float value
Cannot set gray non-stroke color because /'P280' is an invalid float value
Cannot set gray non-stroke color because /'P286' is an invalid float value
Cannot set gray non-stroke color because /'P292' is an invalid float value
Cannot set gray non-stroke color because /'P299' is an invalid float value
Cannot set gray non-stroke color because /'P305' is an invalid float value
Cannot set gray non-stroke color because /'P311' is an invalid float value
Cannot set gray non-strok

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

encoder = OpenAIEncoder(name="text-embedding-3-small")

In [25]:
chunker = SemanticChunker(
        embeddings=OpenAIEmbeddings(),
        breakpoint_threshold_type="percentile",
        breakpoint_threshold_amount=0.85
    )

In [26]:
docs_content = [''.join(doc.page_content) for doc in docs]

In [27]:
chunked_docs = chunker.create_documents(docs_content)

In [28]:
chunked_docs

[Document(metadata={}, page_content='ANNUAL REPORT\n\nOF THE UNIVERSITY\n\nFOR THE YEAR 2023-2024\n\nNATIONAL UNIVERSITY of Computer & Emerging Sciences\n\nFAST House, Rohtas Road, G-9/4\n\nISLAMABAD, PAKISTAN\n\nURL www.nu.edu.pk\n\nEmail registrar@nu.edu.pk\n\nPage 1 of 85\n\nContents Preamble ................................................................................................................... 3\n\nVision .........................................................................................................................'),
 Document(metadata={}, page_content='4\n\nMission ......................................................................................................................'),
 Document(metadata={}, page_content='4\n\nThe Foundation ........................................................................................................'),
 Document(metadata={}, page_content='5\n\nThe University ........................................................

In [29]:
dense_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

In [31]:
url = ""
api_key = ""

In [ ]:
qdrant = QdrantVectorStore.from_documents(
    chunked_docs,
    dense_embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    url=url,
    prefer_grpc=True,
    api_key=api_key,
    collection_name="financial_chunks_contextulized",
)